In [8]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import json
from sklearn.preprocessing import LabelEncoder

In [9]:
# data_file = "iris.json"
# # Load and prepare data
# data = load_iris()
# X = data.data
# y = data.target
# feature_names = list(data.feature_names)
# target_names = list(data.target_names)

# # PCA setup 
# step = 0.05

In [10]:
df = pd.read_csv ("test_dataset.csv")
df.dropna(inplace=True)
data_file = 'fishing.json'

target_names = [
    "Poor Session", "Below Average", "Average Session", "Above Average", 
    "Good Session", "Great Session", "Excellent Session", 
    "Outstanding", "Legendary", "Epic"
]
feature_names = ['engine_age', 'length', 'power', 'month', 'weight', 'y_month',
       'year', 'surf_temp']

#remove non numerical data 
# value adjusted for inflation (check dataset page for more info)
# other non relevant/not known features 
df.drop(["landing", "patch", "value_cpi", "y_", "ID", "dist", "patch_area", "weight_lym", "weight_lm", "val_lm", "val_lym", "nao_index", "price"], axis=1, inplace=True)

y = df["value"]
X = df[feature_names]

# Split the values into 10 categories with meaningful labels
y = pd.cut(y, bins=10, labels=target_names)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Convert categories to numerical values

# PCA setup 
step = 0.2 

In [11]:
# Standardize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [12]:
# Split the data and train the tree (following your original script)
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)
dt_classifier_pca = DecisionTreeClassifier(random_state=42)  # Removed max_depth constraint
dt_classifier_pca.fit(X_train_pca, y_train)

# Generate decision boundary data with finer grid
x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))

In [13]:
# Get predictions for the grid
Z = dt_classifier_pca.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Get prediction probabilities for more detailed visualization
Z_prob = dt_classifier_pca.predict_proba(np.c_[xx.ravel(), yy.ravel()])
Z_prob = Z_prob.reshape((xx.shape[0], xx.shape[1], -1))

In [14]:
# Create grid points and predictions for the frontend
grid_points = []
for i in range(len(xx)):
    for j in range(len(xx[0])):
        grid_points.append({
            'x': float(xx[i][j]),
            'y': float(yy[i][j]),
            'class': int(Z[i][j]),
        })

with open (data_file, "w") as f:
    json.dump (
        {
            "pcaData": X_pca.tolist(),
            "targets": y.tolist(),
            "targetNames": list(target_names),
            "decisionBoundary": {
                "points": grid_points,
                "xRange": [float(x_min), float(x_max)],
                "yRange": [float(y_min), float(y_max)],
                "step": step
            },
        },
        f,
        indent=4
    )